# User2Vec
Based on: https://ieeexplore.ieee.org/document/8875952/

How it works:
- Run doc2vec
- Average vector representations for each user


In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import eland as ed

from src.models import User2Vec, tokenize

# uncomment this if database is not already open (and give ES a couple minutes to set up)
#!make database

ed_df = ed.read_es('localhost', 'twitter')

In [ ]:
df = ed_df[[
    'tweet_id', 'original_tweet_id_str', 
    'user_id', 'name', 'full_text_processed', 
]].to_pandas().fillna(np.nan)

In [ ]:
unique_docs = df_network['full_text_processed'].unique()
train_corpus = [tokenize(doc, tag) for doc, tag in zip(unique_docs, range(unique_docs.shape[0]))]

In [ ]:
model = User2Vec(vector_size=vector_size, min_count=min_count, epochs=epochs)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
vec_cols = [f'vec_{i}' for i in range(vector_size)]

users_id, user_vectors = model.infer_user_vectors(
    df_network['user_id'], 
    df_network['full_text_processed']
)

df_user_vecs = pd.DataFrame(user_vectors, columns=vec_cols)
df_user_vecs['user_id'] = users_id